### Step 1: Set up the workspace

In [2]:
import json

#GETTING MY ID:
id = open('id_confidential.json', 'r')
mi = json.load(id)
my_id = mi["azure_id"]

from azureml.core import Workspace

ws = Workspace.create(name="Class2",
                      subscription_id = my_id,
                      resource_group = "__hw1__",
                      location = "centralindia")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="random_forest_model.pkl",
                                  model_name=mname,
                                  workspace=ws)

Deploying KeyVault with name class2keyvaultb36e944d96.
Deploying AppInsights with name class2insightsd5847d9c8b.
Deployed AppInsights with name class2insightsd5847d9c8b. Took 9.76 seconds.
Deploying StorageAccount with name class2storage2ee2949f4d3.
Deployed KeyVault with name class2keyvaultb36e944d96. Took 26.09 seconds.
Deploying Workspace with name Class2.
Deployed StorageAccount with name class2storage2ee2949f4d3. Took 29.54 seconds.
Deployed Workspace with name Class2. Took 40.67 seconds.
Registering model model


In [3]:
################ score.py ###################
scorepy = f"""import json
import pandas as pd
import joblib
from azureml.core.model import Model

def init():
    global model
    global selected_columns  # Declare selected_columns as global

    # Load the model
    model_path = Model.get_model_path('model')
    model = joblib.load(model_path)

    # Load the selected columns from the JSON file
    selected_columns_path = Model.get_model_path('selected_columns.json')  # Ensure the JSON file is registered
    with open(selected_columns_path, 'r') as f:
        selected_columns = json.load(f)

def run(raw_data):
    try:
        # Parse input data
        data = json.loads(raw_data)['data'][0]
        data = pd.DataFrame(data)

        # Drop selected columns
        data_dummies = data[selected_columns]

        # Perform prediction
        result = model.predict(data_dummies).tolist()
        return json.dumps(result)
    except Exception as e:
        return json.dumps(str(e)).dumps(str(e))


"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

### Step 2: Inference configuration

In [ ]:
from azureml.core.environment import Environment
virtual_env = Environment("env-4-bankrupt")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn','joblib', 'numpy']) ## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\blast\AppData\Local\Temp\ipykernel_38496\354274730.py:16: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [5]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 7270fd3f-8769-4157-b5a8-0187297124e8
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'numpy._core', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'numpy._core', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 7270fd3f-8769-4157-b5a8-0187297124e8
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'numpy._core', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'numpy._core', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: 7270fd3f-8769-4157-b5a8-0187297124e8\nMore information can be found using '.get_logs()'\nError:\n{\n 

In [12]:
print(service.get_logs())

None
